# Plan Semilla 3.0 - Artificial Intelligence Module

<h1><center><span style="color:teal">Search algorithms
</span></center></h1>

<h4><center>William David Moreno Rendon 
<br>
Juan Camilo Sarmiento Peñuela 
<br>
Andres Felipe Rincon Prieto
<br>
Manuel Felipe Arias Pineda
<br>
Yulisa Niño Meza

</center></h4>

# State Space Search

In this challenge you will implement some search algorithms for navigation problems.

Your task is to create an autonomous vehicle that goes from point A to point B. You will be given some grid configurations that will simulate aerial images.

The symbols that form the grid have a special meaning as they specify the type of the terrain and the cost to enter a grid cell with that type of terrain:


In [13]:
import numpy as np

print('Symbol', '   Meaning       ', 'Cost')
print('🛣', '       Street        ', 1)
print('🌲', '       Forest        ', 10)
print('🚸', '       School street ', 2)
print('🚦', '       Traffic light ', 3)
print('🚔', '       Police        ', 4)
print('🚘', '       Traffic       ', 6)
print('🛑', '       Stop sign     ', 5)
print('🚧', '       Construction  ', 7)
print('🏢', '       Buildings     ', 15)

COSTS = {'🛣': 1, '🌲': 10, '🚸': 2, '🚦': 3, '🚔': 4, '🚘': 6, '🛑': 5, '🚧': 7, '🏢': 15}

Symbol    Meaning        Cost
🛣        Street         1
🌲        Forest         10
🚸        School street  2
🚦        Traffic light  3
🚔        Police         4
🚘        Traffic        6
🛑        Stop sign      5
🚧        Construction   7
🏢        Buildings      15


In [14]:
test_map_1 = np.array([['🛣', '🛣', '🛣', '🛣'],
                       ['🛣', '🌲', '🌲', '🛣'],
                       ['🛣', '🌲', '🌲', '🛣'],
                       ['🛣', '🛣', '🛣', '🛣'],
                       ['🌲', '🛣', '🛣', '🛣']])

map1 = np.array([['🛣', '🚸', '🛣', '🛣', '🛣', '🛣', '🛣', '🛑', '🛣', '🛣', '🚦', '🛣', '🛣', '🛣', '🚔', '🛣'],
                 ['🛣', '🌲', '🌲', '🛣', '🛣', '🌲', '🌲', '🛣', '🛣', '🚘', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣'],
                 ['🛣', '🛣', '🛣', '🛣', '🛣', '🌲', '🌲', '🛣', '🛣', '🚘', '🛣', '🌲', '🌲', '🛣', '🚘', '🛣'],
                 ['🛣', '🛑', '🛣', '🚸', '🛣', '🌲', '🛣', '🛣', '🏢', '🛣', '🛣', '🌲', '🌲', '🛣', '🛣', '🚦'],
                 ['🌲', '🚘', '🚘', '🛣', '🛣', '🛣', '🚘', '🛣', '🏢', '🛣', '🛣', '🌲', '🛣', '🛑', '🛣', '🛣'],
                 ['🛣', '🛣', '🛣', '🏢', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣', '🚦', '🛣', '🛣', '🛣', '🛣', '🚸'],
                 ['🛣', '🛣', '🏢', '🏢', '🏢', '🛣', '🛑', '🛣', '🛣', '🛣', '🚘', '🛣', '🛣', '🚸', '🛣', '🛣'],
                 ['🚦', '🛣', '🏢', '🚔', '🛣', '🛣', '🏢', '🚧', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣', '🚔'],
                 ['🛣', '🛣', '🛣', '🚔', '🛣', '🛣', '🏢', '🏢', '🏢', '🚸', '🛣', '🛑', '🚘', '🚔', '🛣', '🛣'],
                 ['🛣', '🛑', '🛣', '🛣', '🛣', '🛣', '🏢', '🏢', '🏢', '🛣', '🛣', '🏢', '🛣', '🛣', '🛣', '🚘'],
                 ['🛣', '🛣', '🛣', '🌲', '🌲', '🚦', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣', '🚧', '🚘'],
                 ['🚘', '🚸', '🛣', '🌲', '🌲', '🛣', '🛣', '🛣', '🛣', '🚔', '🛣', '🛣', '🚦', '🚘', '🛣', '🛣'],
                 ['🚘', '🛣', '🚧', '🛣', '🛣', '🛣', '🛑', '🛣', '🚘', '🚘', '🛣', '🚘', '🛣', '🛣', '🛣', '🛣']])

In [15]:


class Agent:

    def __init__(self, world, costs, initial_state, goal_state):

        self.moves = np.array([[0,1], [0,-1], [-1,0], [1,0]])
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.current_state = initial_state
        self.world = world
        self.costs = costs

    def possible_moves(self):

        succ_states, succ_actions, succ_costs = [],[],[]

        for move in self.moves:

            # check that the next state is in the range of the world
            #print(self.current_state)
            if 0 <= self.current_state[0] + move[0] < self.world.shape[0] and 0 <= self.current_state[1] + move[1] < self.world.shape[1]:

                # move and assign new_state to the position after moving
                new_state = [self.current_state[0] + move[0], self.current_state[1] + move[1]]

                action = [move[0], move[1]]

                cost = self.costs[self.world[new_state[0], new_state[1]]]

                succ_states.append(new_state)
                succ_actions.append(action)
                succ_costs.append(cost)

        return succ_states, succ_actions, succ_costs


    def set_current_state(self, nex_state):

        self.current_state = nex_state



class Node:

    def __init__(self, state, parent=None, action=None, cost=None):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        #self.cost = cost

        self.depth = 0
        self.path_cost = 0

        if parent:
            self.path_cost = parent.path_cost + cost
            self.depth = parent.depth + 1


    def successors(self, agent):

        succ_states, succ_actions, succ_costs = agent.possible_moves()
        children = []

        for i in range(len(succ_states)):
            children.append(Node(succ_states[i], self, succ_actions[i], succ_costs[i]))
        return  children

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        # Team init        
        while node:
            path_back.append(node)
            node = node.parent 
        # Team final
        return list(reversed(path_back))


def depth_first_graph_search(agent):
    """
    Search the deepest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Does not get trapped by loops.
    If two paths reach a state, only use the first one.
    """
    frontier = [(Node(agent.initial_state))]  # Stack
    explored = []
    cont=0
    while frontier:
        node = frontier.pop() # LIFO
        if node.state == agent.goal_state:
            return node.path()
        cont=cont+1
        if(cont==40):
            break
        explored.append(node)
        print("\nNode: ",node)
        print("\ninitial_state",agent.initial_state)
        print("\nCosto: ",agent.costs)
        print("\ncurrent_state: ",node.state)
        print("\ngoal_state",agent.goal_state)
        agent.set_current_state(node.state)
        #agent.current_state=node.state
        for child in node.successors(agent):
            if child not in explored and child not in frontier:
                frontier.append(child)
    return None


def depth_first_graph_search_2(agent):
    #TODO
    """
    Search the deepest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Does not get trapped by loops.
    If two paths reach a state, only use the first one.
    """
    frontier = [(Node(agent.initial_state))]  # Stack
    # frontier_co=[frontier[0].state]
    explored = []
    while frontier:
        # print("stack:",frontier)
        # print("stack",frontier_co)
        node=frontier.pop()    #while there's elements in the stack, pop top item and compare
        # node_co=frontier_co.pop()
        # print("popped stack:",frontier)
        # print("popped stack:",frontier_co)
        agent.set_current_state(node.state)
        # print("agent.current_state",agent.current_state)
        # print("node.state",node.state)
        # print("node.parent",node.parent)
        # print("node.depth",node.depth)
        # print("node.action",node.action)
        # print("node.path_cost",node.path_cost)
        if node.state==agent.goal_state:
            # print("FOUND IT!!!!!")
            return node
        if node.state not in explored:
            explored.append(node.state)
        # print("explored",explored)
        succesors=node.successors(agent)
        # for i in succesors:
        #     print(f"succesor:",i.state) 
        # print("succesors",succesors)
        for item in succesors:
            if item.state not in explored and item.state not in [n.state for n in frontier]: #item not in frontier:
                frontier.append(item)
                # frontier_co.append(item.state)
        # print("stack",frontier)
        # print("stack_co",frontier_co)
        # print("\n")
        # if node.depth>15:
        #     break

    return None


def breadth_first_graph_search_2(agent):
    #TODO
    """
    Search the deepest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Does not get trapped by loops.
    If two paths reach a state, only use the first one.
    """
    frontier = [(Node(agent.initial_state))]  # Queue
    # frontier_co=[frontier[0].state]  #for debugging to view coordinates

    explored = []
    while frontier:
        # print("stack:",frontier)   # for debugging: returns queue with objects and pointers to obj
        # print("stack",frontier_co)   # for debugging: returns queue with states (coordinates)
        # print("stack reversed",list(reversed(frontier)))
        node=frontier.pop(0)    #while there's elements in the stack, pop top item and compare
        # node_pop=frontier_co.pop(0)
        # print("pop stack:",frontier)   #for debugging: prints queue after popping first element
        agent.set_current_state(node.state)
        # print("agent.current_state",agent.current_state)   #for debugging
        # print("node.state",node.state)
        # print("node.parent",node.parent)
        # print("node.depth",node.depth)
        # print("node.action",node.action)
        # print("node.path_cost",node.path_cost)
        if node.state==agent.goal_state:
            # print("FOUND IT!!!!!")
            return node
        if node not in explored:
            explored.append(node.state)
        # print("explored",explored)   #for debugging: returns already visited coordinates
        succesors=node.successors(agent)
        # print("succesors",succesors)   #for debugging: list of node obj with possible succesors
        # for i in succesors:
        #     print(f"succesor:",i.state)   #for debugging: print coordinates of possible succesors
        for item in succesors:   #check each item if its on queue or explored list
            if item.state not in explored and item.state not in [n.state for n in frontier]: #and item not in frontier:
                frontier.append(item)
                # frontier_co.append(item.state)
        # print("stack",frontier)
        # print("stack",frontier_co)
        # print("\n")
        # if node.depth>15:
        #     break

    return None


def breadth_first_graph_search(agent):
    """
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Does not get trapped by loops.
    If two paths reach a state, only use the first one.
    """
    frontier = [Node(agent.initial_state)]  # Queue

    explored = []
    while frontier:
        node = frontier.pop(0)
        if node.state == agent.goal_state:
            return node.path()
        explored.append(node.state)
        print("\nNode: ",node)
        print("\ninitial_state",agent.initial_state)
        print("\nCosto: ",agent.costs)
        print("\ncurrent_state: ",node.state)
        print("\ngoal_state",agent.goal_state)
        #agent.current_state=node.state
        agent.set_current_state(node.state)
        for child in node.successors(agent):
            if child.state not in explored and child.state not in [n.state for n in frontier]:
                    frontier.append(child)

    return None


def uniform_cost_search(agent):
    
    """
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Does not get trapped by loops.
    If two paths reach a state, only use the first one.
    """
    frontier = [(Node(agent.initial_state))]  # PriorityQueue

    explored = []
    while frontier:
        node = frontier.pop(0)
        if agent.goal_state == node.state:
            print("FOUND IT!!!!!")
            print(node.state)
            return node
        
        agent.current_state=node.state
        if node.state not in explored:
            explored.append(node.state)
            children = node.successors(agent)

            for child in children:
                index = 0
                for i in range(len(frontier)):
                    if child.path_cost < frontier[i].path_cost:
                        index = i + 1
                    else:
                        break
                frontier.insert(index, child)

    return None


In [16]:
import numpy as np

world = np.array([[0, 1, 3],
                  [2, 3, 2],
                  [0, 1, 3]])

costs = {0: 10, 1: 1, 2: 5, 3:100}

initial_state = [0, 0]
# goal_state = [2,2]
goal_state=[12,15]

agent1 = Agent(map1, COSTS, initial_state, goal_state)

"""
print(agent1.world)
print(agent1.costs)
print(agent1.initial_state)
print(agent1.goal_state)
print(agent1.moves)
print(agent1.possible_moves())
"""

path = depth_first_graph_search_2(agent1)
path


In [17]:
path.state

[12, 15]

In [18]:
cont=0
for node in path.path():
    cont=cont+1
    print(node.state)

print(cont)

[0, 0]
[1, 0]
[2, 0]
[3, 0]
[4, 0]
[5, 0]
[6, 0]
[7, 0]
[8, 0]
[9, 0]
[10, 0]
[11, 0]
[12, 0]
[12, 1]
[12, 2]
[11, 2]
[10, 2]
[9, 2]
[8, 2]
[7, 2]
[6, 2]
[5, 2]
[4, 2]
[3, 2]
[2, 2]
[1, 2]
[0, 2]
[0, 3]
[0, 4]
[1, 4]
[2, 4]
[3, 4]
[4, 4]
[5, 4]
[6, 4]
[7, 4]
[8, 4]
[9, 4]
[10, 4]
[11, 4]
[12, 4]
[12, 5]
[12, 6]
[11, 6]
[10, 6]
[9, 6]
[8, 6]
[7, 6]
[6, 6]
[5, 6]
[4, 6]
[3, 6]
[2, 6]
[1, 6]
[0, 6]
[0, 7]
[0, 8]
[1, 8]
[2, 8]
[3, 8]
[4, 8]
[5, 8]
[6, 8]
[7, 8]
[8, 8]
[9, 8]
[10, 8]
[11, 8]
[12, 8]
[12, 9]
[12, 10]
[11, 10]
[10, 10]
[9, 10]
[8, 10]
[7, 10]
[6, 10]
[5, 10]
[4, 10]
[3, 10]
[2, 10]
[1, 10]
[0, 10]
[0, 11]
[0, 12]
[1, 12]
[2, 12]
[3, 12]
[4, 12]
[5, 12]
[6, 12]
[7, 12]
[8, 12]
[9, 12]
[10, 12]
[11, 12]
[12, 12]
[12, 13]
[12, 14]
[12, 15]
100


In [19]:
moves = {'01':'⏩','0-1':'⏪','-10':'⏫', '10':'⏬'}

map_copy = map1.copy()

p=path.path()

for i in range(len(p)-1):

    ac = p[i]
    n = p[i+1]

    map_copy[ac.state[0], ac.state[1]] = moves[str(n.action[0])+str(n.action[1])]


for i in range(map_copy.shape[0]):
    print(map_copy[i,:])

['⏬' '🚸' '⏩' '⏩' '⏬' '🛣' '⏩' '⏩' '⏬' '🛣' '⏩' '⏩' '⏬' '🛣' '🚔' '🛣']
['⏬' '🌲' '⏫' '🛣' '⏬' '🌲' '⏫' '🛣' '⏬' '🚘' '⏫' '🛣' '⏬' '🛣' '🛣' '🛣']
['⏬' '🛣' '⏫' '🛣' '⏬' '🌲' '⏫' '🛣' '⏬' '🚘' '⏫' '🌲' '⏬' '🛣' '🚘' '🛣']
['⏬' '🛑' '⏫' '🚸' '⏬' '🌲' '⏫' '🛣' '⏬' '🛣' '⏫' '🌲' '⏬' '🛣' '🛣' '🚦']
['⏬' '🚘' '⏫' '🛣' '⏬' '🛣' '⏫' '🛣' '⏬' '🛣' '⏫' '🌲' '⏬' '🛑' '🛣' '🛣']
['⏬' '🛣' '⏫' '🏢' '⏬' '🛣' '⏫' '🛣' '⏬' '🛣' '⏫' '🛣' '⏬' '🛣' '🛣' '🚸']
['⏬' '🛣' '⏫' '🏢' '⏬' '🛣' '⏫' '🛣' '⏬' '🛣' '⏫' '🛣' '⏬' '🚸' '🛣' '🛣']
['⏬' '🛣' '⏫' '🚔' '⏬' '🛣' '⏫' '🚧' '⏬' '🛣' '⏫' '🛣' '⏬' '🛣' '🛣' '🚔']
['⏬' '🛣' '⏫' '🚔' '⏬' '🛣' '⏫' '🏢' '⏬' '🚸' '⏫' '🛑' '⏬' '🚔' '🛣' '🛣']
['⏬' '🛑' '⏫' '🛣' '⏬' '🛣' '⏫' '🏢' '⏬' '🛣' '⏫' '🏢' '⏬' '🛣' '🛣' '🚘']
['⏬' '🛣' '⏫' '🌲' '⏬' '🚦' '⏫' '🛣' '⏬' '🛣' '⏫' '🛣' '⏬' '🛣' '🚧' '🚘']
['⏬' '🚸' '⏫' '🌲' '⏬' '🛣' '⏫' '🛣' '⏬' '🚔' '⏫' '🛣' '⏬' '🚘' '🛣' '🛣']
['⏩' '⏩' '⏫' '🛣' '⏩' '⏩' '⏫' '🛣' '⏩' '⏩' '⏫' '🚘' '⏩' '⏩' '⏩' '🛣']


In [20]:
initial_state = [0, 0]
# goal_state = [2,2]

agent2 = Agent(map1, COSTS, initial_state, goal_state)

"""
print(agent1.world)
print(agent1.costs)
print(agent1.initial_state)
print(agent1.goal_state)
print(agent1.moves)
print(agent1.possible_moves())
"""

path2 = breadth_first_graph_search_2(agent2)
path2

In [21]:
map_copy = map1.copy()

p2=path2.path()

for i in range(len(p2)-1):

    ac = p2[i]
    n = p2[i+1]

    map_copy[ac.state[0], ac.state[1]] = moves[str(n.action[0])+str(n.action[1])]


for i in range(map_copy.shape[0]):
    print(map_copy[i,:])

['⏩' '⏩' '⏩' '⏩' '⏩' '⏩' '⏩' '⏩' '⏩' '⏩' '⏩' '⏩' '⏩' '⏩' '⏩' '⏬']
['🛣' '🌲' '🌲' '🛣' '🛣' '🌲' '🌲' '🛣' '🛣' '🚘' '🛣' '🛣' '🛣' '🛣' '🛣' '⏬']
['🛣' '🛣' '🛣' '🛣' '🛣' '🌲' '🌲' '🛣' '🛣' '🚘' '🛣' '🌲' '🌲' '🛣' '🚘' '⏬']
['🛣' '🛑' '🛣' '🚸' '🛣' '🌲' '🛣' '🛣' '🏢' '🛣' '🛣' '🌲' '🌲' '🛣' '🛣' '⏬']
['🌲' '🚘' '🚘' '🛣' '🛣' '🛣' '🚘' '🛣' '🏢' '🛣' '🛣' '🌲' '🛣' '🛑' '🛣' '⏬']
['🛣' '🛣' '🛣' '🏢' '🛣' '🛣' '🛣' '🛣' '🛣' '🛣' '🚦' '🛣' '🛣' '🛣' '🛣' '⏬']
['🛣' '🛣' '🏢' '🏢' '🏢' '🛣' '🛑' '🛣' '🛣' '🛣' '🚘' '🛣' '🛣' '🚸' '🛣' '⏬']
['🚦' '🛣' '🏢' '🚔' '🛣' '🛣' '🏢' '🚧' '🛣' '🛣' '🛣' '🛣' '🛣' '🛣' '🛣' '⏬']
['🛣' '🛣' '🛣' '🚔' '🛣' '🛣' '🏢' '🏢' '🏢' '🚸' '🛣' '🛑' '🚘' '🚔' '🛣' '⏬']
['🛣' '🛑' '🛣' '🛣' '🛣' '🛣' '🏢' '🏢' '🏢' '🛣' '🛣' '🏢' '🛣' '🛣' '🛣' '⏬']
['🛣' '🛣' '🛣' '🌲' '🌲' '🚦' '🛣' '🛣' '🛣' '🛣' '🛣' '🛣' '🛣' '🛣' '🚧' '⏬']
['🚘' '🚸' '🛣' '🌲' '🌲' '🛣' '🛣' '🛣' '🛣' '🚔' '🛣' '🛣' '🚦' '🚘' '🛣' '⏬']
['🚘' '🛣' '🚧' '🛣' '🛣' '🛣' '🛑' '🛣' '🚘' '🚘' '🛣' '🚘' '🛣' '🛣' '🛣' '🛣']


In [22]:
cont=0
for node in path2.path():
    cont=cont+1
    print(node.state)

print(cont)

[0, 0]
[0, 1]
[0, 2]
[0, 3]
[0, 4]
[0, 5]
[0, 6]
[0, 7]
[0, 8]
[0, 9]
[0, 10]
[0, 11]
[0, 12]
[0, 13]
[0, 14]
[0, 15]
[1, 15]
[2, 15]
[3, 15]
[4, 15]
[5, 15]
[6, 15]
[7, 15]
[8, 15]
[9, 15]
[10, 15]
[11, 15]
[12, 15]
28


In [23]:
agent3 = Agent(map1, COSTS, initial_state, goal_state)
path3=uniform_cost_search(agent3)
path3

FOUND IT!!!!!
[12, 15]


In [24]:
map_copy = map1.copy()

p3=path3.path()

for i in range(len(p3)-1):

    ac = p3[i]
    n = p3[i+1]

    map_copy[ac.state[0], ac.state[1]] = moves[str(n.action[0])+str(n.action[1])]


for i in range(map_copy.shape[0]):
    print(map_copy[i,:])

['⏩' '⏬' '🛣' '🛣' '🛣' '🛣' '🛣' '🛑' '🛣' '🛣' '🚦' '🛣' '🛣' '🛣' '🚔' '🛣']
['🛣' '⏩' '⏬' '🛣' '🛣' '🌲' '🌲' '🛣' '🛣' '🚘' '🛣' '🛣' '🛣' '🛣' '🛣' '🛣']
['🛣' '🛣' '⏬' '🛣' '🛣' '🌲' '🌲' '🛣' '🛣' '🚘' '🛣' '🌲' '🌲' '🛣' '🚘' '🛣']
['🛣' '🛑' '⏬' '🚸' '🛣' '🌲' '🛣' '🛣' '🏢' '🛣' '🛣' '🌲' '🌲' '🛣' '🛣' '🚦']
['⏬' '⏪' '⏪' '🛣' '🛣' '🛣' '🚘' '🛣' '🏢' '🛣' '🛣' '🌲' '🛣' '🛑' '🛣' '🛣']
['⏬' '🛣' '🛣' '🏢' '🛣' '🛣' '🛣' '🛣' '🛣' '🛣' '🚦' '🛣' '🛣' '🛣' '🛣' '🚸']
['⏬' '🛣' '🏢' '🏢' '🏢' '🛣' '🛑' '🛣' '🛣' '🛣' '🚘' '🛣' '🛣' '🚸' '🛣' '🛣']
['⏬' '🛣' '🏢' '🚔' '🛣' '🛣' '⏩' '⏬' '🛣' '🛣' '🛣' '🛣' '🛣' '🛣' '🛣' '🚔']
['⏬' '🛣' '🛣' '🚔' '🛣' '🛣' '⏫' '⏬' '⏩' '⏬' '🛣' '🛑' '🚘' '🚔' '🛣' '🛣']
['⏩' '⏬' '🛣' '🛣' '🛣' '🛣' '⏫' '⏩' '⏫' '⏬' '🛣' '🏢' '🛣' '🛣' '🛣' '🚘']
['🛣' '⏬' '🛣' '⏩' '⏬' '🚦' '⏫' '🛣' '🛣' '⏬' '🛣' '🛣' '🛣' '🛣' '⏩' '⏬']
['⏬' '⏪' '⏩' '⏫' '⏬' '🛣' '⏫' '🛣' '🛣' '⏬' '🛣' '⏩' '⏩' '⏬' '⏫' '⏬']
['⏩' '⏩' '⏫' '🛣' '⏩' '⏩' '⏫' '🛣' '🚘' '⏩' '⏩' '⏫' '🛣' '⏩' '⏫' '🛣']


In [25]:
cont=0
for node in path3.path():
    cont=cont+1
    print(node.state)

print(cont)

[0, 0]
[0, 1]
[1, 1]
[1, 2]
[2, 2]
[3, 2]
[4, 2]
[4, 1]
[4, 0]
[5, 0]
[6, 0]
[7, 0]
[8, 0]
[9, 0]
[9, 1]
[10, 1]
[11, 1]
[11, 0]
[12, 0]
[12, 1]
[12, 2]
[11, 2]
[11, 3]
[10, 3]
[10, 4]
[11, 4]
[12, 4]
[12, 5]
[12, 6]
[11, 6]
[10, 6]
[9, 6]
[8, 6]
[7, 6]
[7, 7]
[8, 7]
[9, 7]
[9, 8]
[8, 8]
[8, 9]
[9, 9]
[10, 9]
[11, 9]
[12, 9]
[12, 10]
[12, 11]
[11, 11]
[11, 12]
[11, 13]
[12, 13]
[12, 14]
[11, 14]
[10, 14]
[10, 15]
[11, 15]
[12, 15]
56
